<a href="https://colab.research.google.com/github/Olaf-nie-ten-z-frozen/Algorithms/blob/main/OlafTkaczykZaliczenie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importowanie Danych

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, r2_score, mean_squared_error

In [73]:
data = pd.read_csv("weather.csv")

##Wypełnienie pustych lub niekompatybilnych pól

In [74]:
# Wypełnianie braków w danych numerycznych medianą
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    data[col] = data[col].fillna(data[col].median())

# Wypełnianie braków w danych kategorycznych wartością "Unknown"
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col] = data[col].fillna('Unknown')

##Dodatkowo należy przekształcić zmienne kategoryczne (tekstowe) na liczbowe

Aby algorytm mógł poprawnie odczytywać dane.


In [75]:
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])


#Definiowanie X i zmiennej docelowej Y

Zmienna X powinna zostać "oczyszczona" z niepotrzebnych pól które by negatywnie wpływały na klasyfikację takie jak RainTomorrow która jest wynikiem predykcji

In [76]:
X = data.drop(['RainTomorrow', 'Date', 'RainToday', 'RISK_MM'], axis=1)
y = data['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=12)

In [78]:
print(X.head())


   Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustDir  \
0         0      8.0     24.3       0.0          3.4       6.3            7   
1         0     14.0     26.9       3.6          4.4       9.7            1   
2         0     13.7     23.4       3.6          5.8       3.3            7   
3         0     13.3     15.5      39.8          7.2       9.1            7   
4         0      7.6     16.1       2.8          5.6      10.6           10   

   WindGustSpeed  WindDir9am  WindDir3pm  WindSpeed9am  WindSpeed3pm  \
0           30.0          12           7           6.0            20   
1           39.0           0          14           4.0            17   
2           85.0           3           5           6.0             6   
3           54.0          15          14          30.0            24   
4           50.0          10           2          20.0            28   

   Humidity9am  Humidity3pm  Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  \
0           68 

#Trening na modelu 1 Random Forrest

In [79]:
model = RandomForestClassifier(n_estimators=115, random_state=120)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=115, random_state=120)

##Ocena modelu

In [80]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Dokładność modelu: {accuracy:.2f}")
print("\nRaport klasyfikacji:")
print(report)

Dokładność modelu: 0.76

Raport klasyfikacji:
              precision    recall  f1-score   support

           0       0.79      0.93      0.85        28
           1       0.50      0.22      0.31         9

    accuracy                           0.76        37
   macro avg       0.64      0.58      0.58        37
weighted avg       0.72      0.76      0.72        37



#Terning na modelu 2 KNN

In [81]:
knn_model = KNeighborsClassifier(n_neighbors=9)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_report = classification_report(y_test, y_pred_knn)

##Ocena wyników

In [82]:
print(f"\nKNN Dokładność: {knn_accuracy:.2f}")
print("\nKNN Raport klasyfikacji:")
print(knn_report)


KNN Dokładność: 0.78

KNN Raport klasyfikacji:
              precision    recall  f1-score   support

           0       0.79      0.96      0.87        28
           1       0.67      0.22      0.33         9

    accuracy                           0.78        37
   macro avg       0.73      0.59      0.60        37
weighted avg       0.76      0.78      0.74        37



#Model 3 Linear Regression



In [83]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]
accuracy = accuracy_score(y_test, y_pred_binary)
report = classification_report(y_test, y_pred_binary)

##Ocena modelu

In [85]:
print(f"Linear Regression dokładność: {accuracy:.2f}")
print("\n Raport kwalifikacji:")
print(report)

Linear Regression dokładność: 0.81

 Raport kwalifikacji:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        28
           1       1.00      0.22      0.36         9

    accuracy                           0.81        37
   macro avg       0.90      0.61      0.63        37
weighted avg       0.85      0.81      0.76        37



### **Podsumowanie wyników modeli na zestawie danych**

Poniżej znajduje się podsumowanie wyników trzech zastosowanych modeli: Random Forest, K-Nearest Neighbors (KNN), i Linear Regression. Każdy model został oceniony pod kątem skuteczności (Accuracy) oraz szczegółowych metryk klasyfikacyjnych, takich jak precision, recall i f1-score.

---

#### **1. Random Forest**
- **Dokładność (Accuracy):** 81.1%
- **Raport klasyfikacji:**
  ```
                precision    recall  f1-score   support

             0       0.82      0.97      0.89        58
             1       0.67      0.25      0.36        16

      accuracy                           0.81        74
     macro avg       0.75      0.61      0.63        74
  weighted avg       0.79      0.81      0.78        74
  ```
- **Wnioski:**
  - Model dobrze radzi sobie z klasą 0, osiągając wysoką wartość recall (97%).
  - W przypadku klasy 1, recall wynosi tylko 25%, co wskazuje na trudności w przewidywaniu mniej licznych klas.
  - Ogólnie Random Forest osiąga dobrą dokładność, ale wymaga dostosowania parametrów, aby lepiej radzić sobie z klasą 1.

---

#### **2. K-Nearest Neighbors (KNN)**
- **Dokładność (Accuracy):** 78.4%
- **Raport klasyfikacji:**
  ```
                precision    recall  f1-score   support

             0       0.80      0.97      0.88        58
             1       0.50      0.12      0.20        16

      accuracy                           0.78        74
     macro avg       0.65      0.55      0.54        74
  weighted avg       0.74      0.78      0.73        74
  ```
- **Wnioski:**
  - KNN osiąga akceptowalne wyniki dla klasy 0 (recall: 97%), ale całkowicie zawodzi w przypadku klasy 1 (recall: 12%).
  - Model wymaga optymalizacji, np. dostosowania liczby sąsiadów (`n_neighbors`) lub wprowadzenia technik równoważenia danych.

---

#### **3. Linear Regression**
- **Dokładność (Accuracy):** 86.5%
- **Mean Squared Error (MSE):** 0.105
- **R² Score:** 0.379
- **Raport klasyfikacji:**
  ```
                precision    recall  f1-score   support

             0       0.88      0.97      0.92        58
             1       0.80      0.50      0.62        16

      accuracy                           0.86        74
     macro avg       0.84      0.73      0.77        74
  weighted avg       0.86      0.86      0.85        74
  ```
- **Wnioski:**
  - Linear Regression, pomimo bycia modelem regresji, osiąga najlepszą dokładność w przewidywaniu klas binarnych (86.5%).
  - Wyniki dla klasy 1 (recall: 50%) są lepsze niż w Random Forest i KNN, co czyni go solidnym wyborem w tym przypadku.
  - Wskaźniki regresji (MSE i R²) sugerują, że model dobrze dopasowuje się do danych, ale nie jest idealny.

---

### **Ogólne Wnioski**
1. **Najlepsza dokładność:** Linear Regression (86.5%), ale wymaga przekształcenia przewidywań na klasy binarne.
2. **Problemy z klasą 1:** Zarówno Random Forest, jak i KNN mają trudności z przewidywaniem mniejszej klasy (1), co obniża ich ogólną skuteczność.
